In [5]:
import scipy.signal
import numpy
import sys
sys.path.append('..')

In [6]:
import lasp.differential
import lasp.utils
import lasp.filters.linear

In [7]:
arr = numpy.reshape(numpy.arange(0, 16), (4, 4))
arr_fft = numpy.fft.fft2(arr) # img in frequency domain
print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


## dx (with periodic boundary ie circular convolution)

In [8]:
# Derivation
#dx_oper = numpy.array([[ 1, -1 ]]) # operator
kernel2d_id = numpy.pad(numpy.array([[1]]), pad_width=1)
dx_oper = lasp.differential.dx(kernel2d_id)


## With convolution (ie spatial domain)
arr_dx_v1 = scipy.signal.convolve2d(
    arr, 
    dx_oper, 
    mode='same',
    boundary='circular'
)


## With point wise product (THIS IS NOT CONVOLITION CIRCULARY)
### Spatial convolution becomes a point wise product in frequency domain
### For apply a filter in frequency domain, you must padded filter with
### even dimensions that image then you make point wise product in frequency doamin
dx_oper_pad = lasp.utils.pad(dx_oper, arr.shape)
print('padded :\n', dx_oper_pad)
dx_oper_pad_fft = numpy.fft.fft2(dx_oper_pad) # operator in frequency domain
arr_dx_fft_v2 = dx_oper_pad_fft * arr_fft # Point wise product in frequency domain
arr_dx_v2 = numpy.real(numpy.fft.ifft2(arr_dx_fft_v2)) # Return in sptial


# ## BCCB tricks
dx_oper_diag = lasp.utils.fourier_diagonalization(dx_oper, arr.shape)
arr_fft_dx_v3 = dx_oper_diag * arr_fft
arr_dx_v3 = numpy.real(numpy.fft.ifft2(arr_fft_dx_v3))


## With numpy
arr_dx_v4 = lasp.differential.dx(arr)

print('Kernel identity :\n', kernel2d_id)
print('dx_oper :\n', dx_oper)
axis = 1
n= kernel2d_id.shape[axis]
#print('numpy.diff :\n', numpy.transpose(kernel2d_id[:, 0]))
#print('numpy.diff :\n', numpy.diff(kernel2d_id, prepend=numpy.transpose(kernel2d_id[:, 0])))
#print('conv2d :\n', arr_dx_v1)
#print('spatial :\n', arr_dx_v2)
#print('BCCB :\n', arr_dx_v3)
#print('Direct :\n', arr_dx_v4)
#numpy.reshape(kernel2d_id[:, n-1], (-1, 1))
print(arr_dx_v4)


padded :
 [[ 0.  0.  0.  0.]
 [ 0.  1. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
Kernel identity :
 [[0 0 0]
 [0 1 0]
 [0 0 0]]
dx_oper :
 [[ 0.  0.  0.]
 [ 0.  1. -1.]
 [ 0.  0.  0.]]
[[-3.  1.  1.  1.]
 [-3.  1.  1.  1.]
 [-3.  1.  1.  1.]
 [-3.  1.  1.  1.]]


## dy

In [9]:
# Derivation
dy_oper = numpy.transpose(numpy.array([[ 1, -1 ]])) # operator


## With convolution (ie spatial domain)
arr_dy_v1 = scipy.signal.convolve2d(
    arr, 
    dy_oper, 
    mode ='same',
    boundary = 'wrap'
)


## With point wise product
### Spatial convolution becomes a point wise product in frequency domain
### For apply a filter in frequency domain, you must padded filter with
### even dimensions that image then you make point wise product in frequency doamin
dy_oper_pad = lasp.utils.pad(dy_oper, arr.shape)
dy_oper_pad_fft = numpy.fft.fft2(dy_oper_pad) # operator in frequency domain
arr_dy_fft_v2 = dy_oper_pad_fft * arr_fft # Point wise product in frequency domain
arr_dy_v2 = numpy.real(numpy.fft.ifft2(arr_dy_fft_v2)) # Return in sptial


# ## BCCB tricks
dy_oper_diag = lasp.utils.fourier_diagonalization(dy_oper, arr.shape)
arr_fft_dy_v3 = dy_oper_diag * arr_fft
arr_dy_v3 = numpy.real(numpy.fft.ifft2(arr_fft_dy_v3))


## With numpy
arr_dy_v4 = lasp.differential.dy(arr)

print(arr_dy_v1)
print(arr_dy_v2)
print(arr_dy_v3)
print(arr_dy_v4)

[[-12 -12 -12 -12]
 [  4   4   4   4]
 [  4   4   4   4]
 [  4   4   4   4]]
[[-12. -12. -12. -12.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]]
[[  4.   4.   4.   4.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]
 [-12. -12. -12. -12.]]
[[-12. -12. -12. -12.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]
 [  4.   4.   4.   4.]]


## dxT

In [10]:
# Derivation

dx_oper = numpy.array([[ 1, -1 ]]) # operator

## With point wise product
### Spatial convolution becomes a point wise product in frequency domain
### For apply a filter in frequency domain, you must padded filter with
### even dimensions that image then you make point wise product in frequency doamin
dx_oper_pad = lasp.utils.pad(dx_oper, arr.shape)
dx_oper_pad_fft = numpy.fft.fft2(dx_oper_pad)
dxT_oper_pad_fft = numpy.conj(dx_oper_diag)
arr_dxT_fft_v1 = dxT_oper_pad_fft * arr_fft # Point wise product in frequency domain
arr_dxT_v1 = numpy.real(numpy.fft.ifft2(arr_dxT_fft_v1)) # Return in sptial


# ## BCCB tricks
# Derivation
dx_oper_diag = lasp.utils.fourier_diagonalization(dx_oper, arr.shape)
dxT_oper_diag = numpy.conj(dx_oper_diag)
arr_fft_dxT_v2 = dxT_oper_diag * arr_fft
arr_dxT_v2 = numpy.real(numpy.fft.ifft2(arr_fft_dxT_v2))


## With numpy
arr_dxT_v3 = lasp.differential.dxT(arr)


print(arr)
print('Convolution :', 'Not spatial operator')
print('Hadamard in Fourier:\n', arr_dxT_v1)
print('BCCB :\n', arr_dxT_v2)
print('dxT :\n', arr_dxT_v3)



[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
Convolution : Not spatial operator
Hadamard in Fourier:
 [[-1. -1. -1.  3.]
 [-1. -1. -1.  3.]
 [-1. -1. -1.  3.]
 [-1. -1. -1.  3.]]
BCCB :
 [[ 3. -1. -1. -1.]
 [ 3. -1. -1. -1.]
 [ 3. -1. -1. -1.]
 [ 3. -1. -1. -1.]]
dxT :
 [[-1. -1. -1.  3.]
 [-1. -1. -1.  3.]
 [-1. -1. -1.  3.]
 [-1. -1. -1.  3.]]


## dyT

In [11]:
# Derivation

dy_oper = numpy.transpose(numpy.array([[ 1, -1 ]])) # operator

## With point wise product
### Spatial convolution becomes a point wise product in frequency domain
### For apply a filter in frequency domain, you must padded filter with
### even dimensions that image then you make point wise product in frequency doamin
dy_oper_pad = lasp.utils.pad(dy_oper, arr.shape)
dy_oper_pad_fft = numpy.fft.fft2(dy_oper_pad)
dyT_oper_pad_fft = numpy.conj(dy_oper_diag)
arr_dyT_fft_v1 = dyT_oper_pad_fft * arr_fft # Point wise product in frequency domain
arr_dyT_v1 = numpy.real(numpy.fft.ifft2(arr_dyT_fft_v1)) # Return in sptial


# ## BCCB tricks
# Derivation
dy_oper_diag = lasp.utils.fourier_diagonalization(dy_oper, arr.shape)
dyT_oper_diag = numpy.conj(dy_oper_diag)
arr_fft_dyT_v2 = dyT_oper_diag * arr_fft
arr_dyT_v2 = numpy.real(numpy.fft.ifft2(arr_fft_dyT_v2))


## With numpy
arr_dyT_v3 = lasp.differential.dyT(arr)

print('Convolution :', 'Not spatial operator')
print('Hadamard in Fourier:\n', arr_dyT_v1)
print('BCCB :\n', arr_dyT_v2)
print('dxT :\n', arr_dyT_v3)

Convolution : Not spatial operator
Hadamard in Fourier:
 [[12. 12. 12. 12.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]]
BCCB :
 [[12. 12. 12. 12.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]]
dxT :
 [[-4. -4. -4. -4.]
 [-4. -4. -4. -4.]
 [-4. -4. -4. -4.]
 [12. 12. 12. 12.]]


## Laplacian

In [12]:
x_oper = numpy.array([[ 1, -1 ]]) # operator
dx_oper_pad = lasp.utils.pad(dx_oper, arr.shape)
dx_oper_pad_fft = numpy.fft.fft2(dx_oper_pad)
dxT_oper_pad_fft = numpy.conj(dx_oper_diag)

dy_oper = numpy.transpose(numpy.array([[ 1, -1 ]])) # operator
dy_oper_pad = lasp.utils.pad(dy_oper, arr.shape)
dy_oper_pad_fft = numpy.fft.fft2(dy_oper_pad)
dyT_oper_pad_fft = numpy.conj(dy_oper_diag)

lap_pad_fft_v1 = \
    dxT_oper_pad_fft * dx_oper_pad_fft \
    + dyT_oper_pad_fft * dy_oper_pad_fft

arr_lap_fft_v1 = lap_pad_fft_v1 * arr_fft
arr_lap_v1 = numpy.real(numpy.fft.ifft(arr_lap_fft_v1))

###########################

lap_filter = lasp.filters.linear.laplacian()
print('lap filter :\n', lap_filter)

lap_pad_v2 = lasp.utils.pad(lap_filter, arr.shape)
lap_pad_fft_v2 = numpy.fft.fft2(lap_pad_v2)

arr_lap_fft_v2 = lap_pad_fft_v2 * arr_fft
arr_lap_v2 = numpy.real(numpy.fft.ifft2(arr_lap_fft_v2))

###########################

print('\nResults :\n')
print('Lap v1 :\n', arr_lap_v1)
print('lap v2 :\n', arr_lap_v2)

lap filter :
 [[ 0 -1  0]
 [-1  4 -1]
 [ 0 -1  0]]

Results :

Lap v1 :
 [[ 16. -16.   0.   0.]
 [ 16.  16.  16.  16.]
 [ 32.  32.  32.  32.]
 [ 16.  16.  16.  16.]]
lap v2 :
 [[ 20.  12.  16.  16.]
 [-12. -20. -16. -16.]
 [  4.  -4.   0.   0.]
 [  4.  -4.   0.   0.]]


In [13]:
laplacian = lasp.filters.linear.laplacian()
lap_diag_v1 = lasp.utils.fourier_diagonalization(
    kernel = laplacian,
    shape_out = arr.shape 
)

arr_lap_fft_v1 = lap_diag_v1 * arr_fft
arr_lap_v1 = numpy.real(numpy.fft.ifft(arr_lap_fft_v1))

Dx_pad = lasp.utils.pad(numpy.array([[1, -1]]), arr.shape)
Dx = numpy.fft.fft2(Dx_pad)
Dxt = numpy.conj(Dx)

Dy_pad = lasp.utils.pad(numpy.transpose(numpy.array([[1, -1]])), arr.shape)
Dy = numpy.fft.fft2(Dy_pad)
Dyt = numpy.conj(Dy)

Dx_diag = lasp.utils.fourier_diagonalization(
    kernel = Dx_pad,
    shape_out = arr.shape 
)
Dxt_diag = numpy.conj(Dx_diag)

Dy_diag = lasp.utils.fourier_diagonalization(
    kernel = Dy_pad,
    shape_out = arr.shape 
)
Dyt_diag = numpy.conj(Dy_diag)

lap_diag_v2 = Dxt_diag * Dx_diag + Dyt_diag * Dy_diag

arr_lap_fft_v1 = lap_diag_v2 * arr_fft
arr_lap_v1 = numpy.real(numpy.fft.ifft(arr_lap_fft_v2))

###

F2D = Dxt*Dx+Dyt*Dy

print('With laplacian filter :\n', arr_lap_v1)
print('With laplacian definition :\n', arr_lap_v2)
print((F2D == lap_diag_v2))

With laplacian filter :
 [[ 16. -16.   0.   0.]
 [ 16.  16.  16.  16.]
 [ 32.  32.  32.  32.]
 [ 16.  16.  16.  16.]]
With laplacian definition :
 [[ 20.  12.  16.  16.]
 [-12. -20. -16. -16.]
 [  4.  -4.   0.   0.]
 [  4.  -4.   0.   0.]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [14]:
## Test
kernel2d_id = numpy.pad(numpy.array([[1]]), pad_width=1)
dx_oper = lasp.differential.dx(kernel2d_id)
arr_id = scipy.signal.convolve2d(arr, kernel2d_id, mode='same')

arr_dx = scipy.signal.convolve2d(arr, dx_oper, mode='same', boundary='wrap')
print(arr_dx)

[[-3.  1.  1.  1.]
 [-3.  1.  1.  1.]
 [-3.  1.  1.  1.]
 [-3.  1.  1.  1.]]


In [15]:
# def derivation(arr: numpy.ndarray, axis: int) -> numpy.ndarray:
#     shape = numpy.copy(arr.shape)
#     n = shape[axis]
#     taken = numpy.take(arr, axis=axis, indices=n-1)
#     shape[axis] = 1
#     to_add = numpy.reshape(taken, shape)
#     return numpy.diff(arr, prepend=to_add, axis=axis)

In [16]:
def difference_finite_circular(array: numpy.ndarray, axis: int) -> numpy.ndarray:

    prepend = numpy.expand_dims(
        array.take(indices=-1, axis=axis),
        axis=axis
    )

    d_axis = numpy.diff(
        array,
        axis = axis,
        prepend=prepend
    )

    return d_axis

In [17]:
difference_finite_circular(arr, 0)

array([[-12, -12, -12, -12],
       [  4,   4,   4,   4],
       [  4,   4,   4,   4],
       [  4,   4,   4,   4]])

In [18]:
difference_finite_circular(arr, 1)

array([[-3,  1,  1,  1],
       [-3,  1,  1,  1],
       [-3,  1,  1,  1],
       [-3,  1,  1,  1]])

In [99]:
def generic_gradient(arr: numpy.ndarray, axis: int, order: str, mode: str) -> numpy.ndarray:
    tuple_order = (1, 0) if order == 'preorder' else (0, 1)
    padded = numpy.pad(
        array = arr, 
        pad_width = tuple( ( 0, 0 ) if axis != i else tuple_order for i in range(0, arr.ndim) ), 
        mode = mode
    )
    return numpy.diff(a=padded, axis=axis)

def gradient(arr: numpy.ndarray, axis: int, mode: str) -> numpy.ndarray:
    padded = numpy.pad(
        array = arr, 
        pad_width = tuple( ( 0, 0 ) if axis != i else (1, 0) for i in range(0, arr.ndim) ), 
        mode = mode
    )
    return numpy.diff(a=padded, axis=axis)

def transposed_gradient(arr: numpy.ndarray, axis: int, mode: str) -> numpy.ndarray:
    padded = numpy.pad(
        array = arr, 
        pad_width = tuple( ( 0, 0 ) if axis != i else (0, 1) for i in range(0, arr.ndim) ), 
        mode = mode
    )
    return numpy.diff(a=padded, axis=axis)

In [117]:
def Div(u): 
    m,n=u.shape
    div=numpy.zeros((n,m,2))
    # print(div.shape)
    
    #preorder difference 
    div[:-1,:,0]=u[1:,:]-u[:-1,:] #(i+1,j)-(i,j)
    # print(div[:,:,0])
    div[:,:-1,1]=u[:,1:]-u[:,:-1] #(i,j+1)-(i,j)
    # print(div[:,:,1])
    
    #postorder difference 
    div[1:,:,0]=-(div[1:,:,0]-div[:-1,:,0]) #(i,j)-(i-1,j)
    # print(div[:,:,0])
    div[:,1:,1]=-(div[:,1:,1]-div[:,:-1,1]) #(i,j)-(i,j-1)
    # print(div[:,:,1])
    
    div=numpy.sum(div,axis=-1)  #[dx dy].T @ [dx dy] 
    return div

In [118]:
Div(arr)

(4, 4, 2)
[[4. 4. 4. 4.]
 [4. 4. 4. 4.]
 [4. 4. 4. 4.]
 [0. 0. 0. 0.]]
[[1. 1. 1. 0.]
 [1. 1. 1. 0.]
 [1. 1. 1. 0.]
 [1. 1. 1. 0.]]
[[ 4.  4.  4.  4.]
 [-0. -0. -0. -0.]
 [-0. -0. -0. -0.]
 [ 4.  4.  4.  4.]]
[[ 1. -0. -0.  1.]
 [ 1. -0. -0.  1.]
 [ 1. -0. -0.  1.]
 [ 1. -0. -0.  1.]]


array([[5., 4., 4., 5.],
       [1., 0., 0., 1.],
       [1., 0., 0., 1.],
       [5., 4., 4., 5.]])

In [128]:
padded = numpy.pad(arr, ((0, 0), (1, 0)), mode='wrap')
padded[:, 1:] - padded[:, :-1]
# numpy.diff(padded, axis=1)

array([[-3,  1,  1,  1],
       [-3,  1,  1,  1],
       [-3,  1,  1,  1],
       [-3,  1,  1,  1]])

In [129]:
d_arr_dx = grad(arr, axis=1, order='preorder', mode='wrap') # (d / dx) arr
d2_arr_d2x = grad(d_arr_dx, axis=1, order='postorder', mode='wrap') # (d^{2} / d^{2}x) arr
d2_arr_d2x

array([[ 4,  0,  0, -4],
       [ 4,  0,  0, -4],
       [ 4,  0,  0, -4],
       [ 4,  0,  0, -4]])

In [123]:
padded = numpy.pad(arr, ((1, 0), (0, 0)), mode='wrap')
padded[1:, :] - padded[:-1, :]

array([[-12, -12, -12, -12],
       [  4,   4,   4,   4],
       [  4,   4,   4,   4],
       [  4,   4,   4,   4]])

In [130]:
d_arr_dy = grad(arr, axis=0, order='preorder', mode='wrap') # (d / dy) arr
d2_arr_d2y = grad(d_arr_dy, axis=0, order='postorder', mode='wrap') # (d^{2} / d^{2}y) arr
d2_arr_d2y

array([[ 16,  16,  16,  16],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0],
       [-16, -16, -16, -16]])

In [131]:
d2_arr_d2x + d2_arr_d2y

array([[ 20,  16,  16,  12],
       [  4,   0,   0,  -4],
       [  4,   0,   0,  -4],
       [-12, -16, -16, -20]])

In [20]:
def difference_finite_circular(array: numpy.ndarray, axis: int) -> numpy.ndarray:

    prepend = numpy.expand_dims(
        array.take(indices=-1, axis=axis),
        axis=axis
    )

    d_axis = numpy.diff(
        array,
        axis = axis,
        prepend=prepend
    )

    return d_axis

def transposed_difference_finite_circular(array: numpy.ndarray, axis: int) -> numpy.ndarray:
    
    tmp = numpy.flip(array, axis=axis)

    append = numpy.expand_dims(
        tmp.take(indices=0, axis=axis),
        axis=axis
    )

    d_axis = numpy.diff(
        tmp,
        axis = axis,
        append=append
    )

    return d_axis

In [21]:
transposed_difference_finite_circular(arr, 1)

array([[-1, -1, -1,  3],
       [-1, -1, -1,  3],
       [-1, -1, -1,  3],
       [-1, -1, -1,  3]])

In [22]:
arr.ndim

2

In [23]:
class Gradient:

    def __init__(self) -> None:
        pass

    def __call__(self, u: numpy.ndarray) -> numpy.ndarray:

        return numpy.array(
            [
                lasp.differential.difference_finite_circular(u, axis)
                for axis in range(0, u.ndim)
            ]
        )
    
    @classmethod
    def T(cls):
        return TransposedGradient()
    
class TransposedGradient:

    def __init__(self) -> None:
        pass

    def __call__(self, u: numpy.ndarray) -> numpy.ndarray:

        return numpy.array(
            [
                lasp.differential.transposed_difference_finite_circular(u, axis)
                for axis in range(0, u.ndim)
            ]
        )
    
    @classmethod
    def T(cls):
        return Gradient()

In [24]:
grad = Gradient()
grad_vector = grad(arr)
grad_vector

array([[[-12, -12, -12, -12],
        [  4,   4,   4,   4],
        [  4,   4,   4,   4],
        [  4,   4,   4,   4]],

       [[ -3,   1,   1,   1],
        [ -3,   1,   1,   1],
        [ -3,   1,   1,   1],
        [ -3,   1,   1,   1]]])

In [25]:
grad.T()(arr)

array([[[-4, -4, -4, -4],
        [-4, -4, -4, -4],
        [-4, -4, -4, -4],
        [12, 12, 12, 12]],

       [[-1, -1, -1,  3],
        [-1, -1, -1,  3],
        [-1, -1, -1,  3],
        [-1, -1, -1,  3]]])

In [26]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [27]:
numpy.kron(
    numpy.array(
        [
            [5, 10],
            [17, 19]
        ]
    ), numpy.array([[1, 0], [0, 0]]))

array([[ 5,  0, 10,  0],
       [ 0,  0,  0,  0],
       [17,  0, 19,  0],
       [ 0,  0,  0,  0]])

In [28]:
numpy.fft.fft2(
    numpy.kron(
        numpy.array(
            [
                [5, 10],
                [17, 19]
            ]
        ), 
        numpy.array([[1, 0], [0, 0]])
    )
)

array([[ 51.+0.j,  -7.+0.j,  51.+0.j,  -7.+0.j],
       [-21.+0.j,  -3.+0.j, -21.+0.j,  -3.+0.j],
       [ 51.+0.j,  -7.+0.j,  51.+0.j,  -7.+0.j],
       [-21.+0.j,  -3.+0.j, -21.+0.j,  -3.+0.j]])

In [29]:
numpy.kron(
    numpy.array([[1, 1], [1, 1]]),
    numpy.fft.fft2(
        numpy.array(
            [
                [5, 10],
                [17, 19]
            ]
        )
    )
)



array([[ 51.+0.j,  -7.+0.j,  51.+0.j,  -7.+0.j],
       [-21.+0.j,  -3.+0.j, -21.+0.j,  -3.+0.j],
       [ 51.+0.j,  -7.+0.j,  51.+0.j,  -7.+0.j],
       [-21.+0.j,  -3.+0.j, -21.+0.j,  -3.+0.j]])

In [30]:
numpy.fft.fft2( numpy.array([[1, 0], [0, 0]]))

array([[1.+0.j, 1.+0.j],
       [1.+0.j, 1.+0.j]])